# RFs and DL2 (data and MCs) scripts

In this notebook, you can try running magic-cta-pipe on a tiny data sample. Because of time constraints, you could not run the pipeline on all the files needed to produce plots (e.g. SED), so we will provide you a complete dataset to get to 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the MCP scripts 

In [1]:
import glob
import logging
import subprocess
import sys
log = logging.getLogger()

# RFs training

Here you will train RFs: two regressors (energy and direction reconstruction) and one classifier (gamma/hadron separation).

Events are separated according to their combo-type (LST1+M1, LST1+M2, M1+M2, LST1+M1+M2) and these subsamples are used to train telescope-wise RFs. So you will get e.g. 4 classifier output files (4 combo-types) and in each file you will find one RF every telescope 

The script is based on python Scikit-learn.

Options:

-g: MC stereo dl1 gammas directory

-p: MC stereo dl1 protons directory 

-o: directory to save the output

-c: configuration file 

train-\*: to train the energy regressor, the disp regressor or the classifier; you can train one, two or every RF when you launch the script, but you will need all the RFs (energy, disp, classifier) for the next step (Dl1 to Dl2)

use-unsigned: use features absolute value to train RFs 


**Settings**

In [2]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/RF.log','w')

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_dir_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/gamma/train/')
input_dir_proton=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/proton/train/')
output_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school')
config=('/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml')

cd to the scripts directory to launch them

In [3]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Here we use python subprocess.run to run the script and get a log file

In [4]:
a=subprocess.run(['python','lst1_magic_train_rfs.py', f'-g{input_dir_gamma}', f'-p{input_dir_proton}',\
    f'-o{output_dir_rf}', f'-c{config}', '--train-energy', '--train-disp', '--train-classifier',\
        '--use-unsigned'], stdout=f, stderr=f) 

Here you can check your log file

In [5]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/RF.log

# DL1 to DL2 

Here you will use your RFs to get to DL2 MCs and real data: DL2 data contain events whose energy, direction and gammaness are evaluated by RFs telescope-wise (so if three telescopes see the same event, this event will have three energy/direction/gammaness values (LSt1/M1/M2)).

As for RFS, the events are separated in combo_types and the right combo RFs are applied.

Options:

-d: input file (dl1 stereo, MCs or real data)
-r: directory where you stored your RFs
-o: output directory, to store DL2 files



### MCs

**Settings**

In [6]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/DL2_mc.log','w')

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/gamma/test/*.h5')
input_proton=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/proton/test/*.h5')
input_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school/')
output_dir_dl2=('/fefs/aswg/workspace/elisa.visentin/sw_school/')

Lines to get files from gamma/proton folders 

In [7]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()
input_file_proton = glob.glob(input_proton)
input_file_proton.sort()

Here we use python subprocess.run to run the script and get a log file

In [8]:
for input_file in input_file_gamma: 
    b=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)     
        
for input_file in input_file_proton:
    c=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)

Here you can check your log file

In [9]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL2_mc.log

### Real data

**Settings**

In [10]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/DL2_data.log','w')

input_data=('/fefs/aswg/workspace/elisa.visentin/sw_school/DL1_stereo/real/*.h5')
input_dir_rf=('/fefs/aswg/workspace/elisa.visentin/sw_school/')
output_dir_dl2=('/fefs/aswg/workspace/elisa.visentin/sw_school/')

Lines to get files from gamma/proton folders 

In [11]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

Here we use python subprocess.run to run the script and get a log file

In [12]:
for input_file in input_file_data: 
    d=subprocess.run(['python','lst1_magic_dl1_stereo_to_dl2.py', f'-d{input_file}', f'-r{input_dir_rf}',\
        f'-o{output_dir_dl2}'], stdout=f, stderr=f)     

Here you can check your log file

In [13]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL2_data.log